In [39]:
from google.cloud import bigquery
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import pandas as pd

dict_uf = {"11":"Rondônia",
           "12":"Acre",
           "13":"Amazonas",
           "14":"Roraima",
           "15":"Pará",
           "16":"Amapá",
           "17":"Tocantins",
           "21":"Maranhão",
           "22":"Piauí",
           "23":"Ceará",
           "24":"Rio Grande do Norte",
           "25":"Paraíba",
           "26":"Pernambuco",
           "27":"Alagoas",
           "28":"Sergipe",
           "29":"Bahia",
           "31":"Minas Gerais",
           "32":"Espírito Santo",
           "33":"Rio de Janeiro",
           "35":"São Paulo",
           "41":"Paraná",
           "42":"Santa Catarina",
           "43":"Rio Grande do Sul",
           "50":"Mato Grosso do Sul",
           "51":"Mato Grosso",
           "52":"Goiás",
           "53":"Distrito Federal"}

dict_sexo = {'1': 'Homem', '2':'Mulher'}

dict_tipo_moradia = {'1':'Urbana','2':'Rural'}

dict_regiao_moradia = {'1':'Capital', '2':'Região Metropolitana', '4':'Outras regiões'}

dict_cor_raca = {'1':'Branca', '2':'Preta', '3':'Amarela', '4': 'Parda', '5':'Indígena', '9':'Ignorado'}

dict_escolaridade = {'1':'Sem instrução','2':'Fundamental incompleto','3':'Fundamental completo', '4':'Médio incompleto',
                     '5':'Médio completo','6':'Superior incompleto','7':'Superior completo','8':'Pós-graduação'}

dict_trabalhou = {'1':'Sim', '2':'Não', '0':'Não aplicável'}

dict_restricao_contato = {'1':'Sem restrições', '2':'Poucas restrições', '3':'Restrições moderadas', '4':'Muitas restrições', '9':'Sem resposta'}

dict_mes = {'9':'Setembro/2020', '10':'Outubro/2020', '11':'Novembro/2020'}

dict_sintomas = {'1':'Com sintomas', '0':'Sem sintomas'}

dict_procura_medico = {'1':'Procurou atendimento', '2':'Não procurou atendimento'}

dict_internacao = {'1':'Internado', '2':'Não internado', '3':'Não foi atendido', '9':'Sem resposta', '0':'Não se aplica'}

dict_atividade_trabalho = {'10':'Serviços de entregas', '18':'Administração pública','20':'Saúde',
                           '24':'Serviço doméstico','16':'Escritórios','14':'Instituições financeiras','11':'Hospedagem',
                           '15':'Imobiliárias','13':'Informação e comunicação','22':'Artes e esportes','12':'Serviços de alimentação'}

# Conexão com Big Query

In [2]:
project_id = 'brave-tea-400210'
dataset_id = 'fase_3_tech_challenge'
table_id = 'pnad-covid-19'
location = 'US'

client = bigquery.Client(project=project_id, location='US')

# Visualizações

## Distribuição da variável sexo ('A003') nas zonas rurais e urbanas ('V1022')

In [3]:
query_sql = f'''WITH cont_sexo AS (
SELECT
    uf,
    sexo,
    zona_moradia,
    COUNT(sexo) AS n_sexo,
FROM
    `{dataset_id}.{table_id}`
GROUP BY
    uf, zona_moradia, sexo
ORDER BY
    uf
)

SELECT
    uf,
    sexo,
    zona_moradia,
    n_sexo,
    ROUND(SUM(n_sexo)/SUM(n_sexo) OVER (PARTITION BY uf, zona_moradia),4) AS proportion
FROM
    cont_sexo
GROUP BY
    uf, sexo,zona_moradia, n_sexo
ORDER BY uf, sexo
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [4]:
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results['sexo'] = df_results['sexo'].astype(str).replace(dict_sexo)
df_results['zona_moradia'] = df_results['zona_moradia'].astype(str).replace(dict_tipo_moradia)
df_results.head()

,uf,sexo,zona_moradia,n_sexo,proportion
0,Rondônia,Homem,Urbana,5763,0.4784
1,Rondônia,Homem,Rural,2382,0.5205
2,Rondônia,Mulher,Urbana,6283,0.5216
3,Rondônia,Mulher,Rural,2194,0.4795
4,Acre,Homem,Rural,1640,0.5273


In [5]:
fig_urbana = px.bar(df_results.loc[df_results['zona_moradia'] == 'Urbana'].sort_values(by='uf', ascending=False), x="proportion", y="uf", color="sexo", title="Ditribuição da população por sexo (Zona Urbana)",
            color_discrete_sequence=[px.colors.qualitative.G10[1], px.colors.qualitative.G10[0]],height=800, width=600, labels=dict(uf="Estado", sexo="Sexo", proportion="Proporção (%)"))

fig_urbana.update_layout(title_x=0.5, showlegend=False)
fig_urbana.add_vline(x=0.5)


fig_rural = px.bar(df_results.loc[df_results['zona_moradia'] == 'Rural'].sort_values(by='uf', ascending=False), x="proportion", y="uf", color="sexo", title="Ditribuição da população por sexo (Zona Rural)",
            color_discrete_sequence=[px.colors.qualitative.G10[1], px.colors.qualitative.G10[0]],height=800, width=600, labels=dict(uf="Estado", sexo="Sexo", proportion="Proporção (%)"))

fig_rural.update_layout(title_x=0.5, showlegend=False)
fig_rural.add_vline(x=0.5)

fig = make_subplots(rows=1, cols=2, shared_xaxes=True, subplot_titles=["Zona Urbana","Zona Rural"])

# Add the first scatter plot to the first subplot
fig.add_trace(fig_urbana['data'][0], row=1, col=1)
fig.add_trace(fig_urbana['data'][1], row=1, col=1)


# Add the second scatter plot to the second subplot
fig.add_trace(fig_rural['data'][0], row=1, col=2)
fig.add_trace(fig_rural['data'][1], row=1, col=2)
fig.update_layout(barmode='stack', width=1600, height=800, title='Distribuição da população da pesquisa de acordo com zona de moradia e sexo<br>PNAD COVID-19', title_x=0.5)
fig.add_vline(x=0.5)

title_name = fig['layout']['title']['text']
title_name = title_name.replace('<br>PNAD COVID-19','')

plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

fig.show()

Comparando a distribuição de homens e mulheres de acordo com a situação do domicílio (zona urbana ou rural), é perceptível uma tendência de maior presença de mulheres nas zonas urbanas e, por outro lado, uma predominância de homens nas zonas rurais de praticamente todos os estados brasileiros.

Possivelmente, isso está associado às atividades braçais presentes no campo e que são majoritariamente executadas por homens.

Do ponto de vista do COVID-19, esta análise indica que as mulheres estão mais presentes em regiões com grande concentração e circulação de pessoas e por esse motivo podem estar mais suscetíveis à infecção.

# Distribuição da população de acordo com tipo de área (Capital, RM ou Interior)

In [6]:
query_sql =f'''SELECT uf,
            (CASE
                  WHEN regiao_moradia = 1 THEN 1
                  WHEN regiao_moradia = 2 THEN 2
                  WHEN regiao_moradia = 3 THEN 2
                  WHEN regiao_moradia = 4 THEN 4
            END) as new_regiao_moradia,
            COUNT(CASE
                  WHEN regiao_moradia = 1 THEN 1
                  WHEN regiao_moradia = 2 THEN 2
                  WHEN regiao_moradia = 3 THEN 2
                  WHEN regiao_moradia = 4 THEN 4
            END) as n_regiao_moradia

            FROM `{dataset_id}.{table_id}`

            GROUP BY uf, new_regiao_moradia
            ORDER BY uf, new_regiao_moradia
            '''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [7]:
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results['new_regiao_moradia'] = df_results['new_regiao_moradia'].astype(str).replace(dict_regiao_moradia)
df_results.head()

,uf,new_regiao_moradia,n_regiao_moradia
0,Rondônia,Capital,6062
1,Rondônia,Outras regiões,10560
2,Acre,Capital,11498
3,Acre,Outras regiões,7336
4,Amazonas,Capital,16567


In [8]:
fig_3 = px.bar(df_results.sort_values(by=['n_regiao_moradia'], ascending=False), x="uf", y="n_regiao_moradia", color="new_regiao_moradia", title="Distribuição da população da pesquisa por região de moradia<br>PNAD COVID-19",
            labels=dict(uf="Estado", n_regiao_moradia="Nº de entrevistados", new_regiao_moradia="Região de moradia"))

fig_3.update_layout(title_x=0.5, width=1200, height=700, legend_traceorder='reversed')

title_name = fig_3['layout']['title']['text']
title_name = title_name.replace('<br>PNAD COVID-19','')

plotly.offline.plot(fig_3, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

fig_3.show()




# Piramide etária

In [9]:
query_sql = f'''
with idade_data as 
( SELECT 
    sexo,
    idade,
    count(idade) as count_idade
  FROM `{project_id}.{dataset_id}.{table_id}` 
  GROUP BY sexo, idade),

grupo_idade_data as
(
  SELECT
    sexo,
    idade,
    count_idade,
    CASE
      WHEN idade <= 10 THEN '0-10'
      WHEN idade BETWEEN 11 AND 20 THEN '11-20'
      WHEN idade BETWEEN 21 AND 30 THEN '21-30'
      WHEN idade BETWEEN 31 AND 40 THEN '31-40'
      WHEN idade BETWEEN 41 AND 50 THEN '41-50'
      WHEN idade BETWEEN 51 AND 60 THEN '51-60'
      WHEN idade BETWEEN 61 AND 70 THEN '61-70'
      WHEN idade BETWEEN 71 AND 80 THEN '71-80'
      WHEN idade > 80 THEN '80+'
    END as grupo_idade
  FROM idade_data
)

SELECT
  sexo,
  grupo_idade,
  SUM(count_idade) as count_grupo_idade
FROM grupo_idade_data
GROUP BY sexo, grupo_idade
ORDER BY sexo, grupo_idade
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [10]:
df_results['sexo'] = df_results['sexo'].astype(str).replace(dict_sexo)
df_results

,sexo,grupo_idade,count_grupo_idade
0,Homem,0-10,77384
1,Homem,11-20,87847
2,Homem,21-30,75933
3,Homem,31-40,81248
4,Homem,41-50,75747
5,Homem,51-60,68577
6,Homem,61-70,48901
7,Homem,71-80,25158
8,Homem,80+,9861
9,Mulher,0-10,74366


In [48]:
age_groups = list(df_results.sort_values(by='grupo_idade')['grupo_idade'].unique())

males_population = list(df_results.sort_values(by='grupo_idade').loc[df_results['sexo'] == 'Homem']['count_grupo_idade'])
males_population = [-neg_value for neg_value in males_population]
females_population = list(df_results.sort_values(by='grupo_idade').loc[df_results['sexo'] == 'Mulher']['count_grupo_idade'])

# Create a subplot with two horizontal bar charts
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('Homens', 'Mulheres'))

# Add bar traces for males and females
fig.add_trace(go.Bar(x=males_population, y=age_groups, orientation='h', name='Homens'), row=1, col=1)
fig.add_trace(go.Bar(x=females_population, y=age_groups, orientation='h', name='Mulheres'), row=1, col=2)

# Custom xticks
fig.update_layout(xaxis2 = dict(
    tickmode='array',
    tickvals = [90000, 80000, 70000, 60000, 50000, 40000, 30000, 20000, 10000, 0],
    ticktext = ['90,000', '80,000', '70,000', '60,000', '50,000', '40,000', '30,000', '20,000', '10,000', '0']
    )
)

# Custom xticks
fig.update_layout(xaxis1 = dict(
    tickmode='array',
    tickvals = [-90000, -80000, -70000, -60000, -50000, -40000, -30000, -20000, -10000, 0],
    ticktext = ['90,000', '80,000', '70,000', '60,000', '50,000', '40,000', '30,000', '20,000', '10,000', '0']
    )
)

# Customize layout
fig.update_xaxes(title_text='Nº de entrevistados', row=1, col=1)
fig.update_xaxes(title_text='Nº de entrevistados', row=1, col=2)

# Set the range of the y-axis to reverse the age pyramid
#fig.update_yaxes(categoryorder='total ascending', row=1, col=1)
fig.update_yaxes(side='right', mirror=True, row=1, col=1)

# Add title and labels
fig.update_layout(title='Pirâmide etária dos entrevistados<br>PNAD COVID-19', title_x=0.5, xaxis=dict(title='Nº de entrevistados'), width=1000, margin_pad=5,
                  showlegend=False, plot_bgcolor='white')

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.layout.annotations[0].update(y=1.02)
fig.layout.annotations[1].update(y=1.02)

fig.show()

title_name = 'Piramide etária dos entrevistados por sexo'
plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

'./plots/Piramide etária dos entrevistados por sexo.html'

In [40]:
fig.layout.annotations[0].update(y=1.2)

layout.Annotation({
    'font': {'size': 16},
    'showarrow': False,
    'text': 'Homens',
    'x': 0.225,
    'xanchor': 'center',
    'xref': 'paper',
    'y': 1.2,
    'yanchor': 'bottom',
    'yref': 'paper'
})

# Distribuição da cor e raça

In [3]:
query_sql = f'''
with cont_raca as 
(SELECT
  uf,
  cor_raca,
  COUNT(cor_raca) as n_cor_raca
FROM `{project_id}.{dataset_id}.{table_id}` 
GROUP BY uf, cor_raca
ORDER BY uf, cor_raca)

SELECT
    uf,
    cor_raca,
    n_cor_raca,
    ROUND(SUM(n_cor_raca)/SUM(n_cor_raca) OVER (PARTITION BY uf),4) AS proportion_raca
FROM
    cont_raca
GROUP BY
    uf, cor_raca, n_cor_raca
ORDER BY uf, cor_raca
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [8]:
df_results['cor_raca'] = df_results['cor_raca'].astype(str).replace(dict_cor_raca)
df_results['uf'] = df_results['uf'].astype(str).replace(dict_uf)
df_results['proportion_raca'] = df_results['proportion_raca'] * 100
df_results

,uf,cor_raca,n_cor_raca,proportion_raca
0,Rondônia,Branca,4833,29.08
1,Rondônia,Preta,1236,7.44
2,Rondônia,Amarela,145,0.87
3,Rondônia,Parda,10360,62.33
4,Rondônia,Indígena,48,0.29
...,...,...,...,...
153,Distrito Federal,Preta,2071,10.40
154,Distrito Federal,Amarela,195,0.98
155,Distrito Federal,Parda,9734,48.90
156,Distrito Federal,Indígena,51,0.26


In [10]:
fig_cor_raca = px.bar(df_results.sort_values(by=['proportion_raca'], ascending=False), y="uf", x="proportion_raca", color="cor_raca", title="Ditribuição da população da pesquisa por cor/raça<br>PNAD COVID-19",
            labels=dict(uf="Estado", proportion_raca="Proporção da população (%)", cor_raca="Cor/Raça"))

fig_cor_raca.update_layout(title_x=0.5, width=800, height=700)

title_name = 'Distribuição da populacao da pesquisa por cor-raça'

plotly.offline.plot(fig_cor_raca, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

fig_cor_raca.show()


# Relação escolaridade e ocupação

In [3]:
query_sql = f"""
with cont_escolaridade_trabalho as (
SELECT 
  escolaridade, trabalhou, COUNT(*) as n_entrevistados
FROM
  `{project_id}.{dataset_id}.{table_id}`
GROUP BY escolaridade, trabalhou)

SELECT
    escolaridade,
    trabalhou,
    n_entrevistados,
    ROUND(SUM(n_entrevistados)/SUM(n_entrevistados) OVER (PARTITION BY escolaridade),4) AS proportion
FROM
    cont_escolaridade_trabalho
GROUP BY
    escolaridade, trabalhou, n_entrevistados
ORDER BY escolaridade, trabalhou DESC
"""

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [4]:
df_results['escolaridade'] = df_results['escolaridade'].astype(str).replace(dict_escolaridade)
df_results['trabalhou'] = df_results['trabalhou'].astype(str).replace(dict_trabalhou)
df_results = df_results.loc[df_results['trabalhou'] != "Não aplicável"]
df_results.reset_index(drop=True, inplace=True)
df_desemprego = df_results.copy()
n_entrevistados_escolaridade = df_desemprego.groupby('escolaridade').agg({'n_entrevistados':'sum'}).reset_index()
n_entrevistados_escolaridade['n_entrevistados'] = n_entrevistados_escolaridade['n_entrevistados'].astype(int)
display(n_entrevistados_escolaridade)

,escolaridade,n_entrevistados
0,Fundamental completo,75259
1,Fundamental incompleto,264414
2,Médio completo,256040
3,Médio incompleto,107635
4,Pós-graduação,30290
5,Sem instrução,47821
6,Superior completo,107441
7,Superior incompleto,59678


In [5]:
df_desemprego = df_desemprego.loc[df_desemprego['trabalhou'] == 'Sim']
df_desemprego['escolaridade_index'] = list(range(len(df_desemprego['escolaridade'].unique())))
df_desemprego['desemprego'] = (1 - df_desemprego['proportion']) * 100
df_desemprego.drop(columns=['n_entrevistados'],inplace=True)
df_desemprego = df_desemprego.merge(n_entrevistados_escolaridade, on='escolaridade')
df_desemprego

,escolaridade,trabalhou,proportion,escolaridade_index,desemprego,n_entrevistados
0,Sem instrução,Sim,0.0691,0,93.09,47821
1,Fundamental incompleto,Sim,0.2365,1,76.35,264414
2,Fundamental completo,Sim,0.4073,2,59.27,75259
3,Médio incompleto,Sim,0.3309,3,66.91,107635
4,Médio completo,Sim,0.5236,4,47.64,256040
5,Superior incompleto,Sim,0.5184,5,48.16,59678
6,Superior completo,Sim,0.6487,6,35.13,107441
7,Pós-graduação,Sim,0.7469,7,25.31,30290


In [8]:
fig = px.scatter(df_desemprego, x="escolaridade", y="desemprego",
	         size="n_entrevistados", color="escolaridade",
                 hover_name="escolaridade", size_max=60, width=1300, height=600,
                 color_discrete_sequence=px.colors.qualitative.G10,
                 labels=dict(escolaridade="Escolaridade", desemprego="Taxa de desocupação (%)", n_entrevistados="Nº entrevistados"),
                 title="Relação entre escolaridade e taxa de desocupação<br>PNAD COVID-19")

fig.update_yaxes(range=[0, 100])
fig.update_layout(title_x=0.5)

title_name = fig['layout']['title']['text']
title_name = title_name.replace('<br>PNAD COVID-19','')

fig.show()

plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

'./plots/Relação entre escolaridade e taxa de desocupação.html'

# Variação da restrição de contato da população

In [19]:
query_sql = f'''
with cont_restricoes as
(SELECT 
  restricao_contato, mes, COUNT(*) as n_entrevistados
FROM
  `{project_id}.{dataset_id}.{table_id}`
WHERE restricao_contato != 9
GROUP BY mes, restricao_contato
ORDER BY mes)

SELECT
    mes,
    restricao_contato,
    n_entrevistados,
    ROUND(SUM(n_entrevistados)/SUM(n_entrevistados) OVER (PARTITION BY mes),4) AS proportion
FROM
    cont_restricoes
GROUP BY
    mes, restricao_contato,n_entrevistados
ORDER BY mes
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [20]:
df_results['mes'] = df_results['mes'].astype(str).replace(dict_mes)
df_results['restricao_contato'] = df_results['restricao_contato'].astype(str).replace(dict_restricao_contato)

dict_sort = {'Sem restrições':0, 'Poucas restrições':1, 'Restrições moderadas':2, 'Muitas restrições':3,'Sem resposta':4}
df_results.sort_values(by='restricao_contato', key=lambda x: x.map(dict_sort), inplace=True)
df_results.reset_index(drop=True, inplace=True)
df_results

,mes,restricao_contato,n_entrevistados,proportion
0,Setembro/2020,Sem restrições,11191,0.0290
1,Outubro/2020,Sem restrições,17138,0.0452
2,Novembro/2020,Sem restrições,17865,0.0470
3,Setembro/2020,Poucas restrições,148874,0.3863
4,Outubro/2020,Poucas restrições,163684,0.4322
5,Novembro/2020,Poucas restrições,171955,0.4524
6,Setembro/2020,Restrições moderadas,161444,0.4189
7,Outubro/2020,Restrições moderadas,149778,0.3954
8,Novembro/2020,Restrições moderadas,147728,0.3886
9,Setembro/2020,Muitas restrições,63863,0.1657


In [31]:
# Create a line chart using Plotly graph objects
fig = go.Figure()

# Variable 1
fig.add_trace(go.Scatter(
    x=df_results.loc[df_results['mes']=='Setembro/2020']['restricao_contato'], # x-axis
    y=df_results.loc[df_results['mes']=='Setembro/2020']['n_entrevistados'], # y-axis
    mode='lines+markers', # Connect data points with lines and add markers
    name='Setembro/2020', # Name in the legend

    line=dict(
        color=px.colors.qualitative.G10[0],        # Change the line color to red
        width=5),
    
    marker=dict(
        symbol='circle',    # Change the marker symbol to a circle
        size=5,            # Set the marker size
        color=px.colors.qualitative.G10[0]),       # Set the marker color             # Set the line width
))

# Variable 2
fig.add_trace(go.Scatter(
    x=df_results.loc[df_results['mes']=='Outubro/2020']['restricao_contato'], # x-axis
    y=df_results.loc[df_results['mes']=='Outubro/2020']['n_entrevistados'], # y-axis
    mode='lines+markers', # Connect data points with lines and add markers
    name='Outubro/2020', # Name in the legend

    line=dict(
        color=px.colors.qualitative.G10[1],        # Change the line color to red
        width=5),
    
    marker=dict(
        symbol='circle',    # Change the marker symbol to a circle
        size=5,            # Set the marker size
        color=px.colors.qualitative.G10[1]),       # Set the marker color             # Set the line width
))

# Variable 3
fig.add_trace(go.Scatter(
    x=df_results.loc[df_results['mes']=='Novembro/2020']['restricao_contato'], # x-axis
    y=df_results.loc[df_results['mes']=='Novembro/2020']['n_entrevistados'], # y-axis
    mode='lines+markers', # Connect data points with lines and add markers
    name='Novembro/2020', # Name in the legend

    line=dict(
        color=px.colors.qualitative.G10[2],        # Change the line color to red
        width=5),
    
    marker=dict(
        symbol='circle',    # Change the marker symbol to a circle
        size=5,            # Set the marker size
        color=px.colors.qualitative.G10[2]),       # Set the marker color             # Set the line width
))

fig.update_layout(width=1000, height=500, title='Variação das restrições de distanciamento social<br>PNAD COVID-19', title_x=0.5,
                  yaxis_title="Nº de entrevistados", xaxis_title='Nível de restrição',
                  legend=dict(x=0.03,y=0.95,bgcolor='rgba(0,0,0,0)'), autosize=True)

fig.update_yaxes(tickformat=",.0f", range=[0,180000])

fig.show()

title_name = 'Variacaoo restricoes de distanciamento social'

plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

'./plots/Variacaoo restricoes de distanciamento social.html'

In [22]:
dict_sort = {'Sem restrições':0, 'Poucas restrições':1, 'Restrições moderadas':2, 'Muitas restrições':3,'Sem resposta':4}
df_results.sort_values(by='restricao_contato', key=lambda x: x.map(dict_sort))

,mes,restricao_contato,n_entrevistados,proportion
0,Setembro/2020,Sem restrições,11191,0.0289
1,Outubro/2020,Sem restrições,17138,0.0450
2,Novembro/2020,Sem restrições,17865,0.0468
3,Setembro/2020,Poucas restrições,148874,0.3844
4,Outubro/2020,Poucas restrições,163684,0.4302
5,Novembro/2020,Poucas restrições,171955,0.4508
6,Setembro/2020,Restrições moderadas,161444,0.4168
7,Outubro/2020,Restrições moderadas,149778,0.3937
8,Novembro/2020,Restrições moderadas,147728,0.3873
9,Setembro/2020,Muitas restrições,63863,0.1649


# Variação de procura de atendimento entre os sintomáticos

In [50]:
query_sql = f'''
with check_sintomas as (
  SELECT
    mes,
    procurou_medico,
    CASE 
      WHEN tosse = 1 OR febre = 1 OR olfato_paladar = 1 THEN 1
      ELSE 0
    END as sintomas,

    COUNT (CASE 
      WHEN tosse = 1 OR febre = 1 OR olfato_paladar = 1 THEN 1
      ELSE 0
    END) as count_sintomas
  FROM `brave-tea-400210.fase_3_tech_challenge.pnad-covid-19`
  GROUP BY mes, sintomas,procurou_medico
)

SELECT
  mes,
  sintomas,
  procurou_medico,
  count_sintomas,
  ROUND(SUM(count_sintomas)/SUM(count_sintomas) OVER (PARTITION BY mes),4) AS proportion
FROM check_sintomas
WHERE sintomas = 1 AND procurou_medico != 9
GROUP BY mes, sintomas,procurou_medico, count_sintomas
ORDER BY mes
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [51]:
df_results['mes'] = df_results['mes'].astype(str).replace(dict_mes)
df_results['sintomas'] = df_results['sintomas'].astype(str).replace(dict_sintomas)
df_results['procurou_medico'] = df_results['procurou_medico'].astype(str).replace(dict_procura_medico)
df_results

,mes,sintomas,procurou_medico,count_sintomas,proportion
0,Setembro/2020,Com sintomas,Procurou atendimento,2815,0.3756
1,Setembro/2020,Com sintomas,Não procurou atendimento,4680,0.6244
2,Outubro/2020,Com sintomas,Não procurou atendimento,3816,0.6133
3,Outubro/2020,Com sintomas,Procurou atendimento,2406,0.3867
4,Novembro/2020,Com sintomas,Não procurou atendimento,4289,0.6041
5,Novembro/2020,Com sintomas,Procurou atendimento,2811,0.3959


In [52]:
fig = px.bar(df_results, x="mes", y="count_sintomas",
             color='procurou_medico', barmode='group',
             height=600, title='Procura de atendimento médico por entrevistados sintomáticos<br>PNAD COVID-19',
             labels=dict(count_sintomas="Nº de entrevistados", procurou_medico="", mes=''))

fig.update_layout(title_x=0.5)
fig.update_yaxes(tickformat=",.0f")

fig.show()

title_name = 'Procura de atendimento por entrevistados sintomaticos'

plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

'./plots/Procura de atendimento por entrevistados sintomaticos.html'

# Internações por sintoma

In [9]:
sintomas = ['febre','tosse','olfato_paladar']

for sintoma in sintomas:
    query_sql = f'''
        SELECT
            {sintoma},
            internacao,
            COUNT (internacao) as n_entrevistados
        FROM
            `brave-tea-400210.fase_3_tech_challenge.pnad-covid-19`
        WHERE
            {sintoma} = 1 AND internacao != 0 AND internacao != 9
        GROUP BY
            {sintoma}, internacao
        ORDER BY 
            internacao
    '''

    query_job = client.query(query_sql)
    df_results = query_job.to_dataframe()

    df_results.rename(columns={f'{sintoma}':'sintoma'}, inplace=True)
    df_results['sintoma'] = sintoma

    df_results['internacao'] = df_results['internacao'].astype(str).replace(dict_internacao)

    if(sintomas.index(sintoma) == 0):
        final_df = df_results.copy()
    else:
        final_df = pd.concat([final_df,df_results], axis=0)

#final_df['mes'] = final_df['mes'].astype(str).replace(dict_mes)

display(final_df)

,sintoma,internacao,n_entrevistados
0,febre,Internado,296
1,febre,Não internado,2396
2,febre,Não foi atendido,29
0,tosse,Internado,289
1,tosse,Não internado,2458
2,tosse,Não foi atendido,34
0,olfato_paladar,Internado,159
1,olfato_paladar,Não internado,1303
2,olfato_paladar,Não foi atendido,22


In [11]:
final_df['proportion'] = (final_df['n_entrevistados']/final_df.groupby(['sintoma'])['n_entrevistados'].transform('sum'))*100
final_df

,sintoma,internacao,n_entrevistados,proportion
0,febre,Internado,296,10.878354
1,febre,Não internado,2396,88.055862
2,febre,Não foi atendido,29,1.065785
0,tosse,Internado,289,10.391945
1,tosse,Não internado,2458,88.385473
2,tosse,Não foi atendido,34,1.222582
0,olfato_paladar,Internado,159,10.714286
1,olfato_paladar,Não internado,1303,87.803235
2,olfato_paladar,Não foi atendido,22,1.48248


In [8]:
fig = px.histogram(final_df, x="sintoma", y="n_entrevistados",
             color='internacao', barmode='group',
             height=600, title='Número de internados de acordo com tipo de sintoma<br>PNAD COVID-19',
             labels=dict(n_entrevistados="Nº de entrevistados", internacao="", sintoma='Tipo de sintoma'))

fig.update_layout(title_x=0.5)
fig.update_yaxes(tickformat=",.0f")

fig.show()

title_name = 'Numero de internados de acordo com tipo de sintoma'

#plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

# Sintomáticos por atividade trabalho

In [45]:
query_sql = f'''
with check_sintomas as (
  SELECT
    atividade_trabalho,
    CASE 
      WHEN tosse = 1 OR febre = 1 OR olfato_paladar = 1 THEN 1
      ELSE 0
    END as sintomas,

    COUNT (CASE 
      WHEN tosse = 1 OR febre = 1 OR olfato_paladar = 1 THEN 1
      ELSE 0
    END) as count_sintomas
  FROM `brave-tea-400210.fase_3_tech_challenge.pnad-covid-19`
  GROUP BY atividade_trabalho, sintomas
)

SELECT
  atividade_trabalho,
  sintomas,
  count_sintomas,
  ROUND(SUM(count_sintomas)/SUM(count_sintomas) OVER (PARTITION BY atividade_trabalho),4) AS proportion_sintoma
FROM check_sintomas
WHERE atividade_trabalho != 0
GROUP BY
  atividade_trabalho, sintomas, count_sintomas
ORDER BY atividade_trabalho, count_sintomas DESC
'''

query_job = client.query(query_sql)
df_results = query_job.to_dataframe()

In [46]:
df_results = df_results.query('sintomas == 1')
df_results.sort_values(by='proportion_sintoma',ascending=False, inplace=True)
df_results.reset_index(drop=True, inplace=True)
df_results = df_results.loc[:10,:]
df_results['atividade_trabalho'] = df_results['atividade_trabalho'].astype(str).replace(dict_atividade_trabalho)
df_results['sintomas'] = df_results['sintomas'].astype(str).replace(dict_sintomas)
df_results['proportion_sintoma'] = df_results['proportion_sintoma'] * 100
df_results

C:\Users\jacks\AppData\Local\Temp\ipykernel_21676\117334963.py:1: RuntimeWarning:

Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.



,atividade_trabalho,sintomas,count_sintomas,proportion_sintoma
0,Serviços de entregas,Com sintomas,38,2.44
1,Administração pública,Com sintomas,578,2.32
2,Saúde,Com sintomas,669,2.31
3,Serviço doméstico,Com sintomas,515,2.31
4,Escritórios,Com sintomas,269,2.30
5,Instituições financeiras,Com sintomas,133,2.25
6,Hospedagem,Com sintomas,37,2.22
7,Imobiliárias,Com sintomas,48,2.19
8,Informação e comunicação,Com sintomas,168,2.18
9,Artes e esportes,Com sintomas,68,2.11


In [74]:
fig = px.bar(df_results, x="proportion_sintoma", y="atividade_trabalho", orientation='h',
             color_discrete_sequence=px.colors.qualitative.G10, title='Sintomáticos por setor de trabalho<br>PNAD COVID-19',
             labels=dict(atividade_trabalho='', proportion_sintoma='Proporção de sintomáticos (%)'),
             height=400, width=700)

fig.update_layout(yaxis=dict(autorange="reversed"), title_x=0.5)

fig.show()

title_name = 'Sintomaticos por setor de trabalho'

plotly.offline.plot(fig, filename = f'./plots/{title_name}.html', auto_play=False, auto_open=False)

'./plots/Sintomaticos por setor de trabalho.html'